### <center>On The Resemblance and Containment of Documents</center>
#### <center> Andrei Z. Broder </center>

<br>
<br>
<br>
<center>__Presented by Mike Mull (@kwikstep)__</center>

## Introduction

### Resemblance r(A,B)

- r(A,B) is a number between 0 (no resemblance) and 1 (essentially identical).
  - d(A,B) = 1 - r(A,B) is a distance between documents
  
### Containment c(A,B)

- c(A,B) also between 0 and 1.
  - Here 1 means A is entirely contained in B

Broder's intention was to identify similar documents on the web as part of Digital's AltaVista search engine.  He notes that typical string similarity metrics like Levenshtein distance and Hamming distance would require comparing of entire documents, so instead he sets out to define resemblance and containment metrics that can be computed using only a _sketch_ of a document, basically a very abbreviated form that he goes on to define later.

As is usual with similarity metrics, the value of these metrics is defined to go from 0 to 1.  Broder also notes that d = 1 - r is a true _metric_, ie it obeys the triangle inequality, so it's a good option for clustering algorithms.

I'm going to mostly ignore the containment metric, although most of this discussion pertains to both.

## Definitions as set intersection problems
### Shingles (aka, N-Grams)



In [1]:
# N-gram function courtesy of Peter Norvig
def ngrams(seq, n):
    "List all the (overlapping) ngrams in a sequence."
    return (tuple(seq[i:i+n]) for i in range(1+len(seq)-n))

doc1 = "a rose is a rose is a rose"
list(ngrams(doc1.split(), 4))

[('a', 'rose', 'is', 'a'),
 ('rose', 'is', 'a', 'rose'),
 ('is', 'a', 'rose', 'is'),
 ('a', 'rose', 'is', 'a'),
 ('rose', 'is', 'a', 'rose')]

The paper will define the resemblance and containment operation as set operations, but first it goes into detail on the composition of the sets.  Specifically, it defines a _shingle_ (aka, n-gram) as a sequence of overlapping tokens.  The set of interest then is the set of _sequences of tokens_ of length w.  The shingles capture some of the word order in the documents, which is important since the metrics they define are in terms of set intersections and unions, which don't care about order.   

### Resemblance and Containment in Terms of Sets

Suppose S(A,w) is the set of w-shingles for document A

#### Resemblance
$$
r_w(A,B) = \frac{|S(A,w) \cap S(B,w)|}{|S(A,w) \cup S(B,w)|}
$$

#### Containment

$$
c_w(A,B) = \frac{|S(A,w) \cap S(B,w)|}{|S(A,w)|}
$$


The resemblance metric is basically the size of the intersection of two sets of shingles divided by the size of the union. This metrics is also called _Jacccard similarity_. The containment metric is nearly the same except the numerator only includes the size of the set for a single document.

In [2]:
def jaccard_similarity(a, b):
    x = set(a)
    y = set(b)
    return len(x & y) / len(x | y)

print(jaccard_similarity(ngrams('a rose is a rose is a rose'.split(), 1),
                         ngrams('a rose is a flower which is a rose'.split(), 1)))

print(jaccard_similarity(ngrams('a rose is a rose is a rose'.split(), 2),
                         ngrams('a rose is a flower which is a rose'.split(), 2)))

0.6
0.5


Above is a simple implementation of the set resemblance function.  Note that the resemblance is lower for longer shingle lengths.

### Shingles - Option B

In [3]:
blist = set(ngrams(doc1.split(), 4))
blist

{('a', 'rose', 'is', 'a'),
 ('is', 'a', 'rose', 'is'),
 ('rose', 'is', 'a', 'rose')}

Broder defines two methods of creating the set of shingles.  Option B is the simpler (and more usual), however you lose a bit of information since duplicate shingles are collapsed.  Option A, which is shown below keeps an occurence number along with the shingle.

### Shingles - Option A

In [4]:
from collections import defaultdict
counts = defaultdict(int)
alist = []
for t in ngrams(doc1.split(), 4):
    alist.append((t, counts[t]+1))
    counts[t] += 1
    
alist

[(('a', 'rose', 'is', 'a'), 1),
 (('rose', 'is', 'a', 'rose'), 1),
 (('is', 'a', 'rose', 'is'), 1),
 (('a', 'rose', 'is', 'a'), 2),
 (('rose', 'is', 'a', 'rose'), 2)]

## Estimating the resemblance and containment

__Fix a single size w and let &Omega; be the set of all shingles...of size w...__
$$
W \subset \Omega
$$

$$
MIN_{s}(W) = \mbox{the set of smallest s elements in W}
$$


The next section discusses in detail how to _estimate_ the resemblance and containment.  The word estimate is important-- this is a technique based on probability and randomness, so it's not trying to compute these metric exactly.  Broder starts with two definitions.  First, he defines omega, which is the set of all shingles of size w.  As we'll see later the method doesn't really enumerate all possible shingles, but it's important for the descriptions below.

Second, the function MINs(W) is defined, which is just basically the smallest s elements in the set W.  This implies that omega is totally ordered.

### Theorem 1
$$
\pi: \Omega \rightarrow \Omega \ \ \text{(permutation)}
$$

$$
M(A) = MIN_{s}(\pi(S(A,k)))
$$

The paper then proceeds to the main theorem of the paper.  First, they define a permutation pi over the set of shingles, and then they define M(A), which is the minimum s elements of the permutation of the set of shingles for document A.  This function M(A) is what would later be called _MinHash_.  Given these preliminaries, theorem 1 states:

The value

$$
\frac{|MIN_{s}(M(A) \cup M(B)) \cap M(A) \cap M(B)|}{|MIN_{s}(M(A) \cup M(B))|}
$$

is an _unbiased estimator_ of the resemblance of A and B.

This is the key idea in the paper.  I'm sure it's completely self-evident to everyone...just kidding.  First, for completeness let's define _unbiased estimator_.  This is a term from statistics that means that the expected value of this estimator (ie, the average) will approach the true population value as the number of cases gets large.  The important thing here is that this estimator is entirely in terms of the document "sketches".

Now let's break down the parts of this expression. First, we'll look at the union term that's in both the numerator and denominator

$$
\begin{align}
MIN_{s}(M(A) \cup M(B)) & = MIN_{s}(MIN_{s}(\pi(S(A,k)) \cup MIN_s(\pi(S(B,k)))) \\
 &= MIN_{s}(MIN_{s}(\pi(S(A,k)) \cup \pi(S(B,k))) \\
 &= MIN_{s}(\pi(S(A,k)) \cup \pi(S(B,k))) \\
 &= MIN_{s}(\pi(S(A,k)) \cup S(B,k)))
\end{align}
$$


This is a result which is used in the proof of Theorem 1, although i've added some steps to (i hope) make it clearer.  This might still not be obvious, so the code below is a simple example that shows the equivalence in one case:

In [5]:
def pi(x): return set({'a':'x', 'b':'m','c':'t','d':'e', 'e':'r'}[t] for t in x)
def min_s(x, s): return sorted(x)[0:3]

sA = {'a','b','c','e'}
sB = {'a','b','d','e'}

mA = min_s(pi(sA), 3)
mB = min_s(pi(sB), 3)
print(mA, mB)
print(min_s(set(mA) | set(mB), 3))
print(min_s(pi(sA | sB), 3))

['m', 'r', 't'] ['e', 'm', 'r']
['e', 'm', 'r']
['e', 'm', 'r']


- Suppose there are 10 elements in the union of S(A,w) and S(B,w), and 6 elements in the intersection
  - So the resemblance is 0.6
- Another way to think of this is that if you choose an element at random from the union, there's a 60% chance it's also in the intersection

$$
M(A) \cap M(B)
$$

__ Let &alpha; be the smallest element in__ $ \pi(S(A,w)) \cup S(B,w)) $

$$
\begin{align}
Pr(\alpha \in M(A) \cap M(B)) &= Pr(\pi^{-1}(\alpha) \in S(A,w) \cap S(B,w)) \\
&= \frac{|S(A,w) \cap S(B,w)|}{|S(A,w) \cup S(B,w)|}
\end{align}
$$

Now we look at the second part, the intersection of M(A) and M(B).  We choose the minimum element from the previous term and call it &alpha; (actually, we can choose any element of the previous term and this still applies).  What is the probability that &alpha; is in the intersection of M(A) and M(B)?

If an item is in the intersection, then it must be in M(A) __and__ M(B).  That implies that the element before the permutation was in both S(A,w) _and_ S(B,w). Here's the important leap in logic: __this probability is the same as the set resemblance__.


- As before, suppose there are 10 elements in the unions of S(A) and S(B)
- After permuting the union, every element in the union has an equal chance of being the minimum element
- If the resemblance is 0.6 there's 60% chance that the minimum element in the permuted union is also the minimum element in the permuted intersection.
- If it's the mininum element in the permuted intersection, it's the minimum element in both M(A) and M(B)

Again, the important things to note are (1) this is an _estimate_ not an exact value for the resemblance, and (2) we compute the estimate entirely from the document sketches.

- Taking the minimum s elements is effectively like taking a random sample of size s.
   - The larger s is, the closer we get to the true value of the resemblance
   
In later incarnations, Broder changes the methodology so that instead of taking s elements from the permutations, he instead make s permuations and saves the min value from each.  That's more in the spirit of current MinHash implementations.

## Implementation Issues
### Choosing a random permutation and a sample

$$
f : \Omega \rightarrow \{ 0,...,2^l-1 \}
$$


Despite the title of this section it starts with a discussion of basically compressing or hashing the shingles using a function _f_ that will map the shingle to a value between 0 and 2^l in order to reduce storage.  The resemblance is then said to be given by:

$$
r_w(A,B) = \frac{|f(S(A,w)) \cap f(S(B,w))|}{|f(S(A,w)) \cup f(S(B,w))|}
$$


This notation is a bit confusing because in the first part of the section the paper talks about applying _f_ to a shingle, but in this formula f is being applied to a _set_ of shingles.  I'm assuming here that f(S(A,w)) means {f(shingle 1 of A),...,f(shingle w of A)}

__ Fix and arbitrary set $ S \in \Omega $ of size n.  Then if f is chosen uniformly at random__
$$
E(|f(S)|) = 2^l(1-(1-\frac{1}{2^l})^n) = n - \binom{n}{2}\frac{1}{2^l} + \binom{n}{3}\frac{1}{2^{2l}} + \dots
$$

$$
E(|f(S)|) \approx n - \binom{n}{2}\frac{1}{2^l}
$$

The first equation is related to a variation of the _birthday problem_.  This version gives the expected value (mean) of the number of distinct values when choosing n values from a set of size 2<sup>l</sup>.  I won't go into the derivation of this, but here's a [good reference](http://www.math.uah.edu/stat/urn/Birthday.html).

In the second equation Broder is basically saying that we can ignore the possibility of more than 2 shingles being mapped to the same value.  The main point that Broder is trying to make here is that there are likely to be few collisions when applying f to the shingles, so that size of the sets is nearly the same as the original shingles and the set resemblance formula will still work.

__Furthermore it can be argued that the size of f(S) is fairly well concentrated...__

__...it is simpler to use Markov inequality__
$$
P(X \ge a) \le \frac{E(X)}{a} \ \ \text{(Markov inequality)}
$$
$$
P(n - |f(S)| > \binom{n}{2} \frac{1}{2^{l-10}}) = \frac{\binom{n}{2}\frac{1}{2^l}}{\binom{n}{2}\frac{1}{2^{l-10}}}
$$


Here Broder is just trying to show that the variation in the size of the set f(S) is fairly small.  Shown above is the Markov inequality, which shows that "_the probability that the number of collisions exceeds $ \binom{n}{2} \frac{1}{2^{l-10}} $_"

### Rabin Fingerprints

 - Represents strings as a polynomial
 - Basically a hash function but with lower collision probability
 - Can be be computed in a rolling/windowed fashion, which works well for the shingles
 

Broder uses the Rabin fingerprint for f, both because the collision probablity is well understoond and because they can be computed efficiently for the shingles.

### The Permutation

__To produce our sample we need a random permutation $\pi : \{0,1,...,2^l\} \rightarrow  \{0,1,...,2^l\} $ __

Broder's a bit vague on how to generate the permutation, other than to say we need random one, or that we can basically treat the application of the Rabin fingerprint function to each shingle _as_ the permutation.  However, he does cover this in detail in a later paper on generating what he calls _min-wise independent_ permutations, which means any element in the original set has an equal probability of being the minimum element in the permutation.

### The size of the sample
$$
p(s,r,\epsilon) = \sum\limits_{s(r-\epsilon)\le k \le s(r+\epsilon)} \binom{s}{k} r^k(1-r)^{s-k}
$$

(Note: i believe there's a type in the original-- second limit should be r + epsilon)

In this section Broder explains how to choose a sample (basically, the 's' in MINs).  The above formula shows how the probability that the resemblance calculated from the samples can be bounded by &epsilon; with some set probability if you choose the sample size to be large enough.

For example, suppose that the true resemblance is 0.7.  If you had a sample of only 10, then the above formula says that the probability of the computed resemblance being between 0.6 and 0.8 is the sum of the binomial probabilities for 6,7, or 8 successes in 10 tries. 

In [6]:
from scipy.stats import binom

rv = binom(10, 0.7)
print(sum(rv.pmf(k) for k in (6,7,8)))

rv = binom(100, 0.7)
print(sum(rv.pmf(k) for k in range(60,80)))

0.7004233215
0.971038739592


Here you can see that a sample size of 100 gives a much higher probability of getting an estimate within the desired limits than a sample of 10.  Broder points out that with millions of documents, there will still be some cases where the sample is not big enough.

### Evaluating Resemblance

- For any given pair, mergesort removing duplicates the already sorted "sketches" and count the number of duplicates in the first s, which is O(s)
- Go through each sketch, and add to an existing cluster if the resemblance is close enough to a "representative sketch", or start a new cluster.
- __It is likely in practice that each fingerprint belongs only to a few distinct clusters..._

Broder describes a fairly standard clustering procedure, but doesn't give a lot of detail.  One of the key ideas is that fingerprints will only belong to a small number of clusters, so that as the number of clusters grows you don't have to check all of them.

### References and Other Sources

- Andrei Z. Broder (1993). "Some applications of Rabin's fingerprinting method"
-  Broder, Andrei Z.; Charikar, Moses; Frieze, Alan M.; Mitzenmacher, Michael (1998), "Min-wise independent permutations"
- Michael O. Rabin (1981). "Fingerprinting by Random Polynomials"
-  Indyk, Piotr.; Motwani, Rajeev. (1998). "Approximate Nearest Neighbors: Towards Removing the Curse of Dimensionality."